### Import Packages

In [1]:
from __future__ import absolute_import, division, print_function
%load_ext autoreload
%autoreload 2

import numpy as np
from time import gmtime, strftime
import random

# TensorFlow and tf.keras
import tensorflow as tf

import aging as age

print(tf.test.gpu_device_name())

# Define directory with matlab files
# direc = 'D:/MLdata/18-01-09d-Exp/Extract Data/'
# direc  = 'D:/Nicholas_ML/18-01-09d-Exp/Extract Data/'
direc = 'D:/Nick/MLdata/'
# Define directory to save model and plots
savedir = 'D:/Nick/MLdata/Model/' + strftime("%Y-%m-%d %H-%M", gmtime()) + '/'

random.seed(135)
np.random.seed(135)

/device:GPU:0


### Import Data

A lot of things are hidden behind the scenes in this code.  This allows for compactness within the Jupyter notebook.

In order to properly import the data, there are several options which can be varied.

In [6]:
# Optimized Data Extractor
crop_size = 750
split_size = crop_size // 3
clip_value = 1
image_grid_size = int(crop_size**2 / split_size**2)

file_num = 0

folders, files = age.seq_file_names(direc = direc, 
                                which_samples = (1,2,3), 
                                num_exps = file_num)

skip_exp = ['ML_block1_Exp265.mat', 
            'ML_block1_Exp272.mat',
           'ML_block1_Exp278.mat',
           'ML_block1_Exp386.mat',
           'ML_block1_Exp388.mat',
           'ML_block1_Exp392.mat']

for skip in skip_exp:
    files = list(filter((skip).__ne__, files))

['19-03-15a-Exp', '19-03-19a-Exp', '19-03-26a-Exp']


In [ ]:
length_index, split_images, label_dic = age.data_extractor(files, 
                     folders, 
                     direc = direc, 
                     crop_size = crop_size, 
                     split_size = split_size,
                     clip_value = clip_value,
                     subtract = True, 
                     log_image = True, 
                     younger = 0)

### Troubleshooting

In [ ]:
# age.intensity_plotter(length_index, label_dic, 
#                       split_images, image_grid_size,
#                      normalize_to_1 = True, normalize_all = True, 
#                       log_plot = False)

In [ ]:
# age.time_plotter(length_index, files, label_dic, 
#                  split_images, image_grid_size, full = False)
# # age.difference(Fs_label, Fn_label, T_label, split_images, length_index, files, image_grid_size)

### Assemble Data

In [ ]:
# train_data, train_labels, train_meta, test_data, test_labels, test_meta = \
#         age.withhold_sqr(
#             split_images = split_images, 
#             ML_labels = ['T'],
#             label_dic = label_dic,
#             image_grid_size = image_grid_size, 
#             log_time = True,
#             cols = crop_size//split_size)

train_data, train_labels, train_meta, test_data, test_labels, test_meta = \
        age.withold_exp(
            split_images = split_images, 
            ML_labels = ['T'],
            label_dic = label_dic,
            log_time = True)

# train_data, train_labels, test_data, test_labels = \
# age.younger(train_data, train_labels, test_data, test_labels, time_cut = 5)

min_val = np.min([np.min(train_data), np.min(test_data)])

#Final Renormilization of Images to be between 0 and 1
train_data = (train_data - min_val)/(1 - min_val)
test_data = (test_data - min_val)/(1 - min_val)

### Build CNN

In [ ]:
image_size = train_data.shape[1]
    
# Create the base model
# https://keras.io/applications/#inceptionv3
base_model = tf.keras.applications.inception_v3.InceptionV3(
                include_top=False, 
                weights='imagenet', 
                input_shape=(image_size,image_size,3))

base_model.trainable = False
    
incep_model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32, (3, 3)),
    tf.keras.layers.LeakyReLU(alpha=0.5),
    tf.keras.layers.Dropout(rate = 0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64),
    tf.keras.layers.LeakyReLU(alpha=0.5),
    tf.keras.layers.Dropout(rate = 0.3),
    tf.keras.layers.Dense(1),
    tf.keras.layers.LeakyReLU(alpha=0.5)
    ])

#custom_model = tf.keras.Sequential([
#        tf.keras.layers.Conv2D(8, kernel_size=(2, 2),
#                       input_shape=(image_size, image_size, 1)),
#        tf.keras.layers.LeakyReLU(alpha=0.5),
#        tf.keras.layers.Dropout(rate = 0.3),
#        tf.keras.layers.Conv2D(16, (3, 3)),
#        tf.keras.layers.LeakyReLU(alpha=0.5),
#        tf.keras.layers.Dropout(rate = 0.3),
#        tf.keras.layers.Flatten(),
#        tf.keras.layers.Dense(32),
#        tf.keras.layers.LeakyReLU(alpha=0.5),
#        tf.keras.layers.Dropout(rate = 0.3),
#        tf.keras.layers.Dense(1),
#        tf.keras.layers.LeakyReLU(alpha=0.5)

### Train CNN

In [ ]:
model, history = age.inception_train(
                    train_data, 
                    train_labels, 
                    test_data, 
                    test_labels,
                    savedir, 
                    incep_model,
                    epochs = 50)

# model, history = age.aion(train_data, train_labels, test_data, test_labels,
#                     savedir, full_model)

In [ ]:
test_predic = model.predict(np.tile(
    test_data.reshape(test_data.shape[0], 
    test_data.shape[1], 
    test_data.shape[2], 
    1), 
        3))
train_predic = model.predict(np.tile(train_data.reshape(
    train_data.shape[0], 
    train_data.shape[1],  
    train_data.shape[2], 
    1),
        3))

### Examine Results

In [ ]:
age.hist_plotter(history, savedir + "Incep_history")

In [ ]:
train_times = train_labels.flatten()
train_pred_times = train_predic.flatten()

# train_shear = train_labels[:,1]
# train_pred_shear = train_predic[:,1]

test_times = test_labels.flatten()
test_pred_times = test_predic.flatten()

# test_shear = test_labels[:,1]
# test_pred_shear = test_predic[:,1]

age.data_plotter(train_times, train_pred_times, savedir + "Incep_Age_train" , 'b')

age.data_plotter(test_times, test_pred_times, savedir + "Incep_Age_test" , 'k')